In [1]:
!git clone https://github.com/jgbeni/BirdPics.git
!mv BirdPics/utils .
!rm -r BirdPics
!ls -l

Cloning into 'BirdPics'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 48 (delta 14), reused 31 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 6.75 MiB | 25.79 MiB/s, done.
Resolving deltas: 100% (14/14), done.
total 8
drwxr-xr-x 1 root root 4096 Nov  4 14:36 sample_data
drwxr-xr-x 2 root root 4096 Nov  5 22:22 utils


In [15]:
import utils.data_preprocessing as dp
import numpy as np
import h5py
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive',force_remount=True)
device = "cuda" if torch.cuda.is_available() else "cpu"

Mounted at /content/drive


In [14]:
dir = '/content/drive/MyDrive/BirdPics'
os.makedirs(dir+'/models',exist_ok=True)

In [4]:
f = h5py.File(dir+'/data/bird_data.hdf5', "r")

In [5]:
X_train,Y_train = f['train']['X'],np.copy(f['train']['Y'])
X_val,Y_val = f['val']['X'],np.copy(f['val']['Y'])

In [6]:
Y_train = dp.prepare_labels(Y_train)
Y_val = dp.prepare_labels(Y_val)

In [7]:
train_dataset = dp.HDF5Dataset(X_train,Y_train,train=True)
val_dataset = dp.HDF5Dataset(X_val,Y_val)

In [8]:
batch_size = 32

train_loader = DataLoader(train_dataset, num_workers=8, batch_size=batch_size, pin_memory=True,
                                                shuffle=True)
val_loader = DataLoader(val_dataset, num_workers=8, batch_size=batch_size, pin_memory=True,
                                                shuffle=True)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Building the model

In [9]:
dataiter = iter(train_loader)
images,labels = next(dataiter)
print(images.shape)
print(labels.dtype)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


torch.Size([32, 3, 224, 224])
torch.float32


In [26]:
conv1 = nn.Conv2d(3,16,3)
pool = nn.MaxPool2d(2,2)
conv2 = nn.Conv2d(16,32,3)
conv3 = nn.Conv2d(32,64,3)
conv4 = nn.Conv2d(64,64,3)

x = conv1(images)
#print(x.shape)
x = pool(x)
#print(x.shape)
x = conv2(x)
#print(x.shape)
x = pool(x)
#print(x.shape)
x = conv3(x)
#print(x.shape)
x = pool(x)
x = conv4(x)
#print(x.shape)
x = pool(x)
print(x.shape)

torch.Size([32, 64, 12, 12])


In [27]:
class CNN_Net(nn.Module):
   def __init__(self, in_channels, num_classes):

       """
       Building blocks of convolutional neural network.

       Parameters:
           * in_channels: Number of channels in the input image (for grayscale images, 1)
           * num_classes: Number of classes to predict. In our problem, 10 (i.e digits from  0 to 9).
       """
       super(CNN_Net, self).__init__()

       # 1st convolutional layer
       self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3)
       # Max pooling layer
       self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
       # 2nd convolutional layer
       self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
       # 3rd convolutional layer
       self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
       # 4rd convolutional layer
       self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
       #Dropout
       self.dropout50 = nn.Dropout(0.5)
       # Fully connected layers
       self.fc1 = nn.Linear(64*12*12, 500)
       self.fc2 = nn.Linear(500,num_classes)

   def forward(self, x):
       """
       Define the forward pass of the neural network.

       Parameters:
           x: Input tensor.

       Returns:
           torch.Tensor
               The output tensor after passing through the network.
       """
       x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = F.relu(self.conv3(x))  # Apply third convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = x.reshape(x.shape[0], -1)  # Flatten the tensor
       x = self.dropout50(x) #Apply 50% Dropout
       x = F.relu(self.fc1(x))            # Apply fully connected layer 1
       x = self.dropout50(x) #Apply 50% Dropout
       x = self.fc2(x) # Fully connected layer 2
       return x
model = CNN_Net(in_channels=3, num_classes=3).to(device)
print(model)

CNN_Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout50): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=3, bias=True)
)


## Training

In [28]:
# Define the loss function
criterion = nn.CrossEntropyLoss().to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001 ,weight_decay=1e-5)

In [29]:
n_total_steps = len(train_loader)
num_epochs = 20

max_val = 0.
PATH = os.path.join(dir,'models/cnn_v1.pth')

train_loss,val_loss = np.zeros(num_epochs,dtype=np.float32),np.zeros(num_epochs,dtype=np.float32)
train_acc,val_acc = np.zeros(num_epochs,dtype=np.float32),np.zeros(num_epochs,dtype=np.float32)

for epoch in range(num_epochs):
    train_correct,train_samples = 0,0
    val_correct,val_samples = 0,0
    for i, (images, labels) in enumerate(tqdm(train_loader)):
        # origin shape: [32, 3, 224, 224] = 32, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)#+nn.L1Loss(outputs,labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_samples += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc[epoch] = 100.0 * train_correct / train_samples
    train_loss[epoch] = loss.item()
    print('train loss $.3f - train acc. %.2f' %(train_loss[epoch],train_acc[epoch]))


    for val_images, val_labels in val_loader:
        val_images = val_images.to(device)
        val_labels = val_labels.to(device)
        outputs = model(val_images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        val_samples += val_labels.size(0)
        val_correct += (predicted == val_labels).sum().item()
    val_acc = 100.0 * val_correct / val_samples
    if val_acc > max_val:
        max_val = val_acc
        torch.save(model.state_dict(), PATH)
    print('Val acc. %.2f' %(val_acc))


        #if (i+1) % 200 == 0:
            #print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print('Finished Training')

  0%|          | 0/623 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  0%|          | 0/623 [00:00<?, ?it/s]


OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/utils/data_preprocessing.py", line 22, in __getitem__
    image_np = np.array(self.imgs[idx].transpose(2,0,1),float)
                        ~~~~~~~~~^^^^^
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/usr/local/lib/python3.12/dist-packages/h5py/_hl/dataset.py", line 840, in __getitem__
    return self._fast_reader.read(args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  